# **Analyse: Räumliche Verteilung der Periodika in digiPress**


## Installation benötigter Bibliotheken

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim # Zur Geokodierung von Orten
from tqdm.notebook import tqdm # Zum Darstellen von Fortschrittsanzeigen
import time # Zum Arbeiten mit Zeit - hier für Pausen bei Geokodierungsabfragen

# folium = Python-Bibliothek für das Erstellen interaktiver Karten
!pip install folium
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

# Modul zur Berechnung geographischer Distanz
from geopy.distance import geodesic

## Laden der Zeitungsliste

In [ ]:
df = pd.read_excel("Zeitungsliste.xlsx") # Einlesen von Excel-Datei
df.head() # Anzeigen erster fünf Zeilen

## Vorbereitung der Daten

### Trennen von Publikationsort und Herausgeber/Verlag

In [ ]:
# Wert der Spalte "Erschienen" (z.B. "Aarau: Sauerländer") in "Publikationsort" und "Herausgeber" trennen (anhand von Doppelpunkt)
df[['Publikationsort', 'Herausgeber']] = df['Erschienen'].str.split(': ', n=1, expand=True) # Trennung an 1. Doppelpunkt, Aufteilen in zwei eigenständige Spalten

#Spalte "Erschienen" löschen
df = df.drop('Erschienen', axis=1)

# Anzeigen der ersten Zeilen des DataFrames
df.head()

### Markierungen entfernen

Einzelne Publikationsorte sind über eckige Klammern als erschlossen markiert, z.B. *[München]*.

In [ ]:
# Löschen eckiger Klammern im "Publikationsort"
df['Publikationsort'] = df['Publikationsort'].str.replace("[","").str.replace("]","")
df

### Nicht bekannte Publikationsorte

Nicht bekannte Publikationsorte sind in digiPress über *[Erscheinungsort nicht ermittelbar]* gekennzeichnet.

In [ ]:
# Für wie viele Zeitungstitel ist der Publikationsort nicht bekannt?

count = df[df['Publikationsort'] == 'Erscheinungsort nicht ermittelbar'].shape[0]
print(f"Anzahl nicht bekannter Publikationsorte: {count}")

Hier - der Einfachkeit halber und da wenige Fälle - Entfernen von Titeln ohne Publikationsort:

In [ ]:
df = df[df["Publikationsort"] != "Erscheinungsort nicht ermittelbar"]

### Trennen mehrerer Publikationsorte

Für einen Titel können mehrere Publikationsorte genannt sein, z.B. "Leipzig, Dresden" - aber Vorsicht: Beistriche müssen nicht unbedingt ein Marker hierfür sein, z.B. "Halle, S." oder "Frankfurt, M".

In [ ]:
# Funktion zur Trennung (mit Berücksichtigung von Ausnahmen)

def split_publikationsort(location_string):
    locations = [loc.strip() for loc in location_string.split(',') if loc.strip()] # Trennung bei Beistrich
    split_locations = [] # Liste zum Sammeln der Orte

    # Überprüfung, ob es sich um Ausnahmen handelt
    for loc in locations:
        # Falls Ort nur einen Buchstaben und optional einen Punkt umfasst:
        if (len(loc) == 1 and loc.isalpha()) or (len(loc) == 2 and loc[0].isalpha() and loc[1] == '.'):
            if split_locations:
                # Zusammenführung mit vorherigem Ort
                split_locations[-1] = split_locations[-1] + ', ' + loc
        else:
            split_locations.append(loc)

    return split_locations

# Beispiel
print(split_publikationsort("Halle, S., Leipzig"))

In [ ]:
# Inhalt der Spalte "Publikationsort" je in Liste umwandeln
df['Publikationsort'] = df['Publikationsort'].apply(split_publikationsort)

# DataFrame entlang dieser Spalte 'explodieren' lassen = für jeden Publikationsort wird eine Zeile angelegt, die anderen Werte werden dupliziert
df = df.explode('Publikationsort').reset_index(drop=True)

# Display the updated DataFrame
df.head()

## Top 10 der Publikationsorte in digiPress

In [ ]:
# Anzahl pro Ort zählen
location_counts = df["Publikationsort"].value_counts()

# 10 häufigste Orte
top_10_locations = location_counts.head(10) # Nummer ändern, falls mehr Werte angezeigt werden sollen

# Display the top 10 locations and their counts
display(top_10_locations)

**Frage:** Wodurch ist das obige Ergebnis verzerrt?

In [ ]:
# Top 10 der Publikationsorte in digiPress - nun auf Ebene des Zeitungsunternehmens

# Jeder Ort pro digiPress-ID soll nur einmal gezählt werden (= .groupby("Publikationsort"))
top_10_locations_by_id = df.groupby('Publikationsort')['digiPress-ID'].nunique().sort_values(ascending=False).head(10)

display(top_10_locations_by_id)

## Kartendarstellung der Publikationsorte

(Hinweis: Titelebene)

### Geokodierung

Die Ortsangaben in den digiPress-Metadaten sind reine 'Strings' (Text) und müssen für die Darstellung auf einer Karte erst an Koordinaten gekoppelt (= geokodiert) werden.

In [ ]:
# Initialisierung von Geocoder geocoder
geolocator = Nominatim(user_agent="nrast", timeout=10) # WICHTIG: persönlichen, beliebigen User-Agent setzen (z.B. Kombination aus Teilen des eigenen Namens)

In [ ]:
# Beispiel zum Testen
location = geolocator.geocode("Köln", exactly_one=True, timeout=10)
print(location, location.latitude, location.longitude)

**Aufgabe**: Testen Sie einige Beispiele. Wie erfolgreich ist die Geokodierung der Publikationsorte über Nominatim, was könnte eine Herausforderung darstellen?

In [ ]:
# Anwendung auf gesamte Daten - Dauer ca. 5 Minuten

# Fortschrittsanzeige
tqdm.pandas()

# Initialisierung von Cache - damit bereits geokodierte Strings (z.B. "Leipzig") nicht jedes Mal neu abgefragt werden
geo_cache = {}

# Hilfsfunktion für Cache
def geocode_city(city):
    if pd.isna(city):
        return pd.Series([None, None])

    # Wenn Ort bereits im Cache: Koordinaten von dort nehmen
    if city in geo_cache:
        return pd.Series(geo_cache[city])

    try:
        location = geolocator.geocode(city, exactly_one=True, timeout=10)
        time.sleep(1)
        if location:
            result = (location.latitude, location.longitude)
        else:
            result = (None, None)
    except Exception as e: # Fehler melden
        print(f"Error for {city}: {e}")
        result = (None, None)

    # Speichern in Cache
    geo_cache[city] = result
    return pd.Series(result)

# Anwendung auf Datensatz
df[["lat_Publikationsort", "lon_Publikationsort"]] = df["Publikationsort"].progress_apply(geocode_city)

df.head()

In [ ]:
# Alternative (falls Geokodierung zu lange dauert): Vorbereite Version laden

df = pd.read_excel("Zeitungsliste_geokodiert.xlsx") # Einlesen von Excel-Datei
df.head() # Anzeigen erster fünf Zeilen

### Visualisierung

In [ ]:
# Erstellen von interaktiver Karte, die auf Mittelpunkt der Daten hinzoomt
map = folium.Map(location=[df["lat_Publikationsort"].mean(), df["lon_Publikationsort"].mean()], zoom_start=4)

# Einschalten von MarkerCluster - viele beeinander liegende Marker werden geclustered
marker_cluster = MarkerCluster().add_to(map)

# Hinzufügen eines Markers für jede Zeile des DataFrames
for idx, row in df.dropna(subset=["lat_Publikationsort", "lon_Publikationsort"]).iterrows():
    folium.Marker(
        location=[row["lat_Publikationsort"], row["lon_Publikationsort"]], # Koordinaten definieren
        popup=f"""Ort: {row['Publikationsort']}<br>Titel: {row["Titel"]}<br>digiPress-ID:{row["digiPress-ID"]}"""
    ).add_to(marker_cluster)

map

**Fragen:**
* Welche Schlüsse können aus der Karte gezogen werden?
* Welche Probleme hat diese Kartenansicht und wie würden sich diese lösen lassen?

## Kartendarstellung der Verbreitungsorte

Die folgende Zelle führt die diversen Schritte nochmal für die Verbreitungsorte durch. Voraussetzung ist, dass die obigen Zellen bereits ausgeführt worden, da auf dort definierte Funktionen etc. zurückgegriffen wird.

### Geokodierung und Visualisierung

In [ ]:
# Ausführung dauert ca. 7-10 Minuten

# Einträge von "Verbreitungsort(e)" an ";" aufteilen (Daten sind konsistent!) - nun eine Zeile pro Kombination aus Publikations- und Verbreitungsort
df2 = (
    df.dropna(subset=["Verbreitungsort(e)"])
      .assign(Verbreitungsort=df["Verbreitungsort(e)"].str.split("; "))
      .explode("Verbreitungsort"))

# Geokodierung
tqdm.pandas()
geo_cache = {}
df2[["lat_Verbreitungsort", "lon_Verbreitungsort"]] = df2["Verbreitungsort"].progress_apply(geocode_city)

#Kartenvisualisierung
map_verbreitung = folium.Map(location=[df2["lat_Verbreitungsort"].mean(), df2["lon_Verbreitungsort"].mean()], zoom_start=4)
marker_cluster = MarkerCluster().add_to(map_verbreitung)
for idx, row in df2.dropna(subset=["lat_Verbreitungsort", "lon_Verbreitungsort"]).iterrows():
    folium.Marker(
        location=[row["lat_Verbreitungsort"], row["lon_Verbreitungsort"]], # lat und lon als Koordinaten verwendet
        popup=f"""Ort: {row['Verbreitungsort']}<br>Titel: {row["Titel"]}<br>digiPress-ID:{row["digiPress-ID"]}"""
    ).add_to(marker_cluster)

map_verbreitung

In [ ]:
# Alternative (falls Geokodierung zu lange dauert): Vorbereitete Daten verwenden

df2 = pd.read_excel("Zeitungsliste_erweitert_geokodiert.xlsx") # Einlesen von Excel-Datei

#Kartenvisualisierung
map_verbreitung = folium.Map(location=[df2["lat_Verbreitungsort"].mean(), df2["lon_Verbreitungsort"].mean()], zoom_start=4)
marker_cluster = MarkerCluster().add_to(map_verbreitung)
for idx, row in df2.dropna(subset=["lat_Verbreitungsort", "lon_Verbreitungsort"]).iterrows():
    folium.Marker(
        location=[row["lat_Verbreitungsort"], row["lon_Verbreitungsort"]], # lat und lon als Koordinaten verwendet
        popup=f"""Ort: {row['Verbreitungsort']}<br>Titel: {row["Titel"]}<br>digiPress-ID:{row["digiPress-ID"]}"""
    ).add_to(marker_cluster)

map_verbreitung

### Alternative: Heatmap

In [ ]:
# Folium für Heatmap verwenden

heatmap = folium.Map(location=[df2["lat_Verbreitungsort"].mean(), df2["lon_Verbreitungsort"].mean()], zoom_start=4)
heat_data = df2.dropna(subset=["lat_Verbreitungsort", "lon_Verbreitungsort"])[["lat_Verbreitungsort", "lon_Verbreitungsort"]].values.tolist()
HeatMap(heat_data, radius=12, blur=15, max_zoom=6).add_to(heatmap)
heatmap

## Bonus: Zeitungen mit der geographisch (!) weitesten Reichweite finden

Beziehungsweise: Finden von Fehlern in der Geokodierung

In [ ]:
# Funktion zur Kalkulation der Distanz zwischen Publikations- und Verbreitungsort (in Kilometer)
def calc_distance(row):
    if pd.notna(row["lat_Verbreitungsort"]) and pd.notna(row["lat_Publikationsort"]):
        return geodesic(
            (row["lat_Verbreitungsort"], row["lon_Verbreitungsort"]),
            (row["lat_Publikationsort"], row["lon_Publikationsort"])
        ).kilometers
    else:
        return None

# Neue Spalte mit Distanz hinzufügen
df2["dist_km"] = df2.apply(calc_distance, axis=1)

# Subset erstellen: 50 Zeilen mit höchster Distanz
top10 = df2.nlargest(50, "dist_km")

# Anzeigen als Dataframe mit ausgewählten Spalten
top10[["Titel", "digiPress-ID", "Verbreitungsort", "Publikationsort", "dist_km"]]

## Speicherung als Excel-Dateien

In [ ]:
df.to_excel("Zeitungsliste_geokodiert.xlsx", index=False)

In [ ]:
df2.to_excel("Zeitungsliste_erweitert_geokodiert.xlsx", index=False)